# Team Ratings

In [1]:
# Local libraries
import Tools.ratings_utils as ru

FILENAME = "Data/data_2024.json"
TOURNAMENT_FILENAME = "Data/tournament_2024.csv"

In [2]:
# Currently unused (save for Massey, Colley, Elo Ratings)
tournament = Tournament.Tournament(url=TOURNAMENT_URL,
                                   debug=True)

NameError: name 'Tournament' is not defined

## Massey Ratings

In [3]:
score_df = ru.set_massey_rating_data_frame(filename=FILENAME)
massey_ratings = ru.calculate_massey_ratings(score_df=score_df,
                                               debug=False)

In [15]:
import pandas as pd

# Load tournament CSV file into a DataFrame
tourney_df = pd.read_csv(TOURNAMENT_FILENAME)

# Display the first 5 rows
print(tourney_df.head())
# print(massey_ratings)

tourney_dict = {
    "Round": [],
    "Game": [],
    "Team1": [],
    "Team2": [],
    "Rating1": [],
    "Rating2": []
}

# Add ratings to 1st round
for i in range(32):
    
    team1 = tourney_df["Team1"][i]
    team2 = tourney_df["Team2"][i]
    rating1 = massey_ratings[team1]
    rating2 = massey_ratings[team2]
    
    tourney_dict["Round"].append(tourney_df["Round"][i])
    tourney_dict["Game"].append(tourney_df["Game"][i])
    tourney_dict["Team1"].append(team1)
    tourney_dict["Team2"].append(team2)
    tourney_dict["Rating1"].append(rating1)
    tourney_dict["Rating2"].append(rating2)

print(pd.DataFrame(tourney_dict).head())

# Compare 1st round scores

   Round Conference  Game             Team1         Team2  Score1  Score2
0      1       East     1       Connecticut       Stetson      91      52
1      1       East     2  Florida Atlantic  Northwestern      65      77
2      1       East     3   San Diego State           UAB      69      65
3      1       East     4            Auburn          Yale      76      78
4      1       East     5     Brigham Young      Duquesne      67      71
   Round  Game             Team1         Team2    Rating1    Rating2
0      1     1       Connecticut       Stetson  45.398735  16.722209
1      1     2  Florida Atlantic  Northwestern  34.476610  33.382178
2      1     3   San Diego State           UAB  35.811758  25.808480
3      1     4            Auburn          Yale  43.980661  25.408055
4      1     5     Brigham Young      Duquesne  40.938876  28.166539
